In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import isnull, avg, min, date_format
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pyspark.sql.functions as F
import pyspark.pandas as ps
import numpy as np
import datetime
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, DoubleType

ps.set_option('compute.ops_on_diff_frames', True)

/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
24/08/20 15:51:44 WARN Utils: Your hostname, admins-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.246.23.126 instead (on interface en0)
24/08/20 15:51:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/20 15:51:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
# TO BE UPDATED
POST_FP = "../EDA/gu/data/아이오닉 누수/clien_posts.csv" # from S3
COMMENT_FP = "../EDA/gu/data/아이오닉 누수/clien_comments.csv" # from S3
MODEL_FP = "../Model/model_result.csv" # from RedShift
OUTPUT_FP = "output/Transform_result.csv" # to RedShift

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('spark-job') \
    .config('spark.executor.memory', '4gb') \
    .config("spark.executor.cores", "4") \
    .getOrCreate()

24/08/20 15:51:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
post_df = spark.read.option("header", True).option("quote", "\"").option("escape", "\"").option("multiline", "true").option("sep", ",").csv(POST_FP, sep=',',).to_pandas_on_spark()
comment_df = spark.read.option("header", True).option("quote", "\"").option("multiline", "true").csv(COMMENT_FP, sep=',').to_pandas_on_spark() 
model_df = spark.read.option("header", True).csv(MODEL_FP, sep=',').to_pandas_on_spark()

# collected_at = datetime.datetime.now().replace(second=0, microsecond=0)
collected_at = ps.to_datetime('2023-10-31 00:00:52').replace(second=0, microsecond=0) # from S3 (crawling)
if collected_at.minute >= 30:
    collected_at = collected_at.replace(minute=30)
else:
    collected_at = collected_at.replace(minute=00)

/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/sql/dataframe.py:5725: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(
/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(


In [5]:
def find_hot_criteria(row, model_df):
    condition = ((model_df['post_type']==1) \
        & (model_df['relative_time']==row.relatvie_time) \
        & (model_df['cumulative_num']==row.comments))
    hot_pdf_value = model_df[condition].pdf.values[0]
    return hot_pdf_value


def find_cold_criteria(row, model_df):
    condition = ((model_df['post_type']==0) \
        & (model_df['relative_time']==row.relatvie_time) \
        & (model_df['cumulative_num']==row.comments))
    hot_pdf_value = model_df[condition].pdf.values[0]
    return hot_pdf_value


def predict_post_type(post_df, model_df):    
    per_post_hot_probability = post_df.apply(lambda x: find_hot_criteria(x, model_df), axis=1)
    per_post_cold_probability = post_df.apply(lambda x: find_cold_criteria(x, model_df), axis=1)
    post_df['impact'] = np.log(per_post_hot_probability/per_post_cold_probability)
    post_df['post_type'] = 0
    post_df.loc[post_df['impact'] > 0, 'post_type'] = 1
    return post_df

def parse_date(date_str):
    return ps.to_datetime(date_str, errors="coerce")

In [6]:
# preprocessing
post_df['created_at'] = post_df['created_at'].apply(parse_date)
comment_df.columns = ['cmt_author', 'cmt_count', 'post_id', 'cmt_created_at']     
comment_df['cmt_created_at'] = comment_df['cmt_created_at'].apply(parse_date)
model_df['pdf'] = model_df['pdf'].astype(float)
model_df['cumulative_num'] = model_df['cumulative_num'].astype(int)
model_df['post_type'] = model_df['post_type'].astype(int)

/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(
/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(


In [7]:
# filtering 
cutoff_time = collected_at - datetime.timedelta(hours=35)
post_df = post_df[post_df['created_at'] > cutoff_time]
comment_df = comment_df[comment_df['cmt_created_at'] > cutoff_time]

# To be deleted (below)
post_df = post_df[post_df['created_at'] <= collected_at] 
comment_df = comment_df[comment_df['cmt_created_at'] <= collected_at]

In [8]:
df = ps.merge(post_df, comment_df, left_on='id', right_on='post_id', how='left')

time_interval = 5 * 60 # seconds 
df['relative_time'] = ((df['cmt_created_at'] - df['created_at']) // time_interval) * 5

/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/pandas/data_type_ops/datetime_ops.py:71: UserWarning: Note that there is a behavior difference of timestamp subtraction. The timestamp subtraction returns an integer in seconds, whereas pandas returns 'timedelta64[ns]'.
  warnings.warn(msg, UserWarning)


In [9]:
df_group_by_post_id = df.groupby('id').agg({'cmt_author': 'count'}).rename(columns={'cmt_author': 'comments'})
df_group_by_post_id = df_group_by_post_id.reset_index()

In [10]:
post_df = ps.merge(post_df, df_group_by_post_id, left_on='id', right_on='id', how='inner')

In [11]:
post_df['relatvie_time'] =  ((collected_at - df['created_at']) // time_interval) * 5

/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/pandas/data_type_ops/datetime_ops.py:98: UserWarning: Note that there is a behavior difference of timestamp subtraction. The timestamp subtraction returns an integer in seconds, whereas pandas returns 'timedelta64[ns]'.
  warnings.warn(msg, UserWarning)


In [12]:
model_df['pdf'] = model_df['pdf'].astype(float)
model_df['cumulative_num'] = model_df['cumulative_num'].astype(int)
model_df['relative_time'] = model_df['relative_time'].astype(int)
model_df['post_type'] = model_df['post_type'].astype(int)

In [13]:
post_df = predict_post_type(post_df, model_df)

/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is

In [14]:
post_df['sns_id'] = 0
post_df['collected_at'] = collected_at

In [15]:
batch_id = int(collected_at.strftime('%Y%m%d%H%M'))
post_df['batch_id'] = batch_id
post_df.rename(columns={'id': 'post_id'}, inplace = True) 

In [18]:
post_df.to_csv(OUTPUT_FP, header=True)

/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
/Users/admin/Desktop/softeer-team-project/spark-job/.spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The a